In [0]:
from google.colab import files
import pandas as pd
import numpy as np 
import keras 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.utils.np_utils import to_categorical 
from keras.utils.np_utils import to_categorical 
from keras.callbacks import ReduceLROnPlateau,LearningRateScheduler

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

 Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Access the file
downloaded1 = drive.CreateFile({'id':'1S3ZuU97EH-_qtq3KNJQEM6uLE9hI_FzI'}) # replace the id with id of file you want to access
downloaded2 = drive.CreateFile({'id':'1erHD9KAWAx7MPyhBnRt7kqkVaHitpaAV'})
downloaded1.GetContentFile('mnist_test.csv')  
downloaded2.GetContentFile('mnist_train.csv') 


import pandas as pd
df_test = pd.read_csv('mnist_test.csv') 
df_train = pd.read_csv('mnist_train.csv')

In [0]:
# We are creating more data by scaling, zooming and shifting the original training images)
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=10, zoom_range = 0.10,  width_shift_range=0.1, height_shift_range=0.1)

In [0]:
# Create 5 CNN ensemble models

## Architecture taken from kaggle kernels
##https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist

## Author : Chris Deotte

n_nets = 5
model = [0] *n_nets
for j in range(n_nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(10, activation='softmax'))

    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
## Prepare datasets
Y_train = df_train["Category"]
X_train = df_train.drop(["Id","Category"],axis = 1) 
X_test = df_test.drop(["Id"],axis = 1) 


In [8]:
X_train.shape

(60000, 784)

In [0]:
# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [9]:
X_train.shape

(60000, 784)

In [0]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = 10)

In [14]:
# Decrease Learning rate as we progress through epochs to improve learning rate
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
# Training phase
fit_mo = [0] * n_nets
epochs = 30
for i in range(n_nets):
    X_train_val, X_val, Y_train_val, Y_val = train_test_split(X_train, Y_train, test_size = 0.1)
    fit_mo[i] = fit_mo[i].fit_generator(datagen.flow(X_train_val,Y_train_val, batch_size=64), ## Generate more images
        epochs = epochs, steps_per_epoch = X_train_val.shape[0]//64,validation_data = (X_val,Y_val), callbacks=[annealer], 
        verbose=2)
   

Epoch 1/30
 - 47s - loss: 0.3973 - acc: 0.8787 - val_loss: 0.0601 - val_acc: 0.9820
Epoch 2/30
 - 36s - loss: 0.1151 - acc: 0.9651 - val_loss: 0.1304 - val_acc: 0.9648
Epoch 3/30
 - 36s - loss: 0.0835 - acc: 0.9750 - val_loss: 0.0382 - val_acc: 0.9885
Epoch 4/30
 - 36s - loss: 0.0734 - acc: 0.9781 - val_loss: 0.0294 - val_acc: 0.9903
Epoch 5/30
 - 36s - loss: 0.0679 - acc: 0.9800 - val_loss: 0.0297 - val_acc: 0.9907
Epoch 6/30
 - 34s - loss: 0.0625 - acc: 0.9810 - val_loss: 0.0267 - val_acc: 0.9930
Epoch 7/30
 - 33s - loss: 0.0530 - acc: 0.9838 - val_loss: 0.0262 - val_acc: 0.9927
Epoch 8/30
 - 33s - loss: 0.0484 - acc: 0.9857 - val_loss: 0.0285 - val_acc: 0.9925
Epoch 9/30
 - 34s - loss: 0.0472 - acc: 0.9858 - val_loss: 0.0233 - val_acc: 0.9933
Epoch 10/30
 - 34s - loss: 0.0425 - acc: 0.9876 - val_loss: 0.0299 - val_acc: 0.9922
Epoch 11/30
 - 35s - loss: 0.0413 - acc: 0.9876 - val_loss: 0.0248 - val_acc: 0.9922
Epoch 12/30
 - 35s - loss: 0.0364 - acc: 0.9892 - val_loss: 0.0218 - val_a

In [0]:
# Add the probability of all the 5 ensemble models and choose the max prob value
results = np.zeros( (X_test.shape[0],10) ) 
for j in range(5):
    results = results + model[j].predict(X_test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Category")
submission = pd.concat([pd.Series(range(1,(X_test.shape[0]+1)),name = "Id"),results],axis = 1)
submission.to_csv("Mnist-Cnn.csv",index=False)

In [0]:
files.download('Mnist-Cnn.csv')